In [1]:
import torch
import torch.nn as nn
import numpy as np

class PointWOLF(object):
    # todo2: delete args
    def __init__(self):
        # self.num_anchor = args.w_num_anchor
        # self.sample_type = args.w_sample_type
        # self.sigma = args.w_sigma

        # self.R_range = (-abs(args.w_R_range), abs(args.w_R_range))
        # self.S_range = (1., args.w_S_range)
        # self.T_range = (-abs(args.w_T_range), abs(args.w_T_range))

        # todo1: set the value to default
        self.num_anchor = 4
        self.sample_type = 'fps'  # 'random'
        self.sigma = 0.5

        self.R_range = (-abs(10), abs(10))
        self.S_range = (1., 3)
        self.T_range = (-abs(0.25), abs(0.25))
        
        
    def __call__(self, pos):
        """
        input :
            pos([N,3])
            
        output : 
            pos([N,3]) : original pointcloud
            pos_new([N,3]) : Pointcloud augmneted by PointWOLF
        """
        M=self.num_anchor #(Mx3)
        N, _=pos.shape #(N)
        
        if self.sample_type == 'random':
            idx = np.random.choice(N,M)#(M)
        elif self.sample_type == 'fps':
            idx = self.fps(pos, M) #(M)
        
        pos_anchor = pos[idx] #(M,3), anchor point
        
        pos_repeat = np.expand_dims(pos,0).repeat(M, axis=0)#(M,N,3)
        # todo
        pos_normalize = np.zeros_like(pos_repeat, dtype=pos.dtype)  #(M,N,3)
        # pos_normalize = np.zeros_like(pos_repeat)  #(M,N,3)
        
        #Move to canonical space
        print('pos_repeat type and shape: ', type(pos_repeat), pos_repeat.shape)
        print('pos_anchor type and shape: ', type(pos_anchor), pos_anchor.shape)
        
        pos_normalize = pos_repeat - pos_anchor.reshape(M,-1,3)
        
        #Local transformation at anchor point
        pos_transformed = self.local_transformaton(pos_normalize) #(M,N,3)
        
        #Move to origin space
        pos_transformed = pos_transformed + pos_anchor.reshape(M,-1,3) #(M,N,3)
        
        pos_new = self.kernel_regression(pos, pos_anchor, pos_transformed)
        pos_new = self.normalize(pos_new)
        
        return pos.astype('float32'), pos_new.astype('float32')
        

    def kernel_regression(self, pos, pos_anchor, pos_transformed):
        """
        input :
            pos([N,3])
            pos_anchor([M,3])
            pos_transformed([M,N,3])
            
        output : 
            pos_new([N,3]) : Pointcloud after weighted local transformation 
        """
        M, N, _ = pos_transformed.shape
        
        #Distance between anchor points & entire points
        sub = np.expand_dims(pos_anchor,1).repeat(N, axis=1) - np.expand_dims(pos,0).repeat(M, axis=0) #(M,N,3), d
        
        project_axis = self.get_random_axis(1)

        projection = np.expand_dims(project_axis, axis=1)*np.eye(3)#(1,3,3)
        
        #Project distance
        sub = sub @ projection # (M,N,3)
        sub = np.sqrt(((sub) ** 2).sum(2)) #(M,N)  
        
        #Kernel regression
        weight = np.exp(-0.5 * (sub ** 2) / (self.sigma ** 2))  #(M,N) 
        pos_new = (np.expand_dims(weight,2).repeat(3, axis=-1) * pos_transformed).sum(0) #(N,3)
        pos_new = (pos_new / weight.sum(0, keepdims=True).T) # normalize by weight
        return pos_new

    
    def fps(self, pos, npoint):
        """
        input : 
            pos([N,3])
            npoint(int)
            
        output : 
            centroids([npoints]) : index list for fps
        """
        N, _ = pos.shape
        centroids = np.zeros(npoint, dtype=np.int_) #(M)
        distance = np.ones(N, dtype=np.float64) * 1e10 #(N)
        farthest = np.random.randint(0, N, (1,), dtype=np.int_)
        for i in range(npoint):
            centroids[i] = farthest
            centroid = pos[farthest, :]
            dist = ((pos - centroid)**2).sum(-1)
            # distance_torch = torch.from_numpy(distance)
            # print('dist type: ', type(dist))
            # print('distance type', type(distance))
            # todo: numpy and torch
            mask = dist < distance
            distance[mask] = dist[mask]
            farthest = distance.argmax()
        return centroids
    
    def local_transformaton(self, pos_normalize):
        """
        input :
            pos([N,3]) 
            pos_normalize([M,N,3])
            
        output :
            pos_normalize([M,N,3]) : Pointclouds after local transformation centered at M anchor points.
        """
        M,N,_ = pos_normalize.shape
        transformation_dropout = np.random.binomial(1, 0.5, (M,3)) #(M,3)
        transformation_axis =self.get_random_axis(M) #(M,3)

        degree = np.pi * np.random.uniform(*self.R_range, size=(M,3)) / 180.0 * transformation_dropout[:,0:1] #(M,3), sampling from (-R_range, R_range) 
        
        scale = np.random.uniform(*self.S_range, size=(M,3)) * transformation_dropout[:,1:2] #(M,3), sampling from (1, S_range)
        scale = scale*transformation_axis
        scale = scale + 1*(scale==0) #Scaling factor must be larger than 1
        
        trl = np.random.uniform(*self.T_range, size=(M,3)) * transformation_dropout[:,2:3] #(M,3), sampling from (1, T_range)
        trl = trl*transformation_axis
        
        #Scaling Matrix
        S = np.expand_dims(scale, axis=1)*np.eye(3) # scailing factor to diagonal matrix (M,3) -> (M,3,3)
        #Rotation Matrix
        sin = np.sin(degree)
        cos = np.cos(degree)
        sx, sy, sz = sin[:,0], sin[:,1], sin[:,2]
        cx, cy, cz = cos[:,0], cos[:,1], cos[:,2]
        R = np.stack([cz*cy, cz*sy*sx - sz*cx, cz*sy*cx + sz*sx,
             sz*cy, sz*sy*sx + cz*cy, sz*sy*cx - cz*sx,
             -sy, cy*sx, cy*cx], axis=1).reshape(M,3,3)
        
        pos_normalize = pos_normalize@R@S + trl.reshape(M,1,3)
        return pos_normalize
    
    def get_random_axis(self, n_axis):
        """
        input :
            n_axis(int)
            
        output :
            axis([n_axis,3]) : projection axis   
        """
        axis = np.random.randint(1,8, (n_axis)) # 1(001):z, 2(010):y, 3(011):yz, 4(100):x, 5(101):xz, 6(110):xy, 7(111):xyz    
        m = 3 
        axis = (((axis[:,None] & (1 << np.arange(m)))) > 0).astype(int)
        return axis
    
    def normalize(self, pos):
        """
        input :
            pos([N,3])
        
        output :
            pos([N,3]) : normalized Pointcloud
        """
        pos = pos - pos.mean(axis=-2, keepdims=True)
        scale = (1 / np.sqrt((pos ** 2).sum(1)).max()) * 0.999999
        pos = scale * pos
        return pos

In [2]:
import torch
import numpy as np
import torch.nn as nn

random_input = np.ones((1024, 3), dtype=np.float32)
wolf = PointWOLF()

In [3]:
original, augmentation = wolf(random_input)
print(original.shape, augmentation.shape)

pos_repeat type and shape:  <class 'numpy.ndarray'> (4, 1024, 3)
pos_anchor type and shape:  <class 'numpy.ndarray'> (4, 3)
(1024, 3) (1024, 3)


In [4]:
# 测试另外一个输入，就是点的个数不同，但是维度相同的输入
random_input_2 = np.ones((2837, 3), dtype=np.float32)
o, n = wolf(random_input_2)
print(o.shape, n.shape)

pos_repeat type and shape:  <class 'numpy.ndarray'> (4, 2837, 3)
pos_anchor type and shape:  <class 'numpy.ndarray'> (4, 3)
(2837, 3) (2837, 3)


没有问题，说明针对输入还是robust的。

发现问题，最好还是把输入设置为numpy。解决
接着需要安装open3d

In [25]:
!pip3 install open3d

     |████████████████████████████████| 127.9 MB 1.9 MB/s eta 0:00:01


现在想要测试一下，给定一个点云的状态下，原始的和Aug生成的区别有多大，是不是在接受的范围之内。
创建可视化函数

In [5]:
import os
import time
import random
import open3d as o3d

In [6]:
import numpy as np

In [7]:
file_path = '/Users/huangqianliang/Desktop/demo/person_0001.txt'

In [8]:
# 采样结果可视化函数
def visualize_samples(d_sample):
    """
    input - numpy.array
    """
    print("Points in downsample set: ", len(d_sample))
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

In [10]:
# 提取并可视化
point_set = np.loadtxt(file_path, delimiter=',').astype(np.float32)[:, 0:3]
point_set.shape

(10000, 3)

In [11]:
point_input = point_set
original, augmentation = wolf(point_input)
print(original.shape, augmentation.shape)


pos_repeat type and shape:  <class 'numpy.ndarray'> (4, 10000, 3)
pos_anchor type and shape:  <class 'numpy.ndarray'> (4, 3)
(10000, 3) (10000, 3)


In [12]:
visualize_samples(augmentation)

Points in downsample set:  10000
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


可以显示，但是存在问题。
首先是启动，然后就直接在命令行里面报错。所以估计还是在Linux下的比较好。
现在需要的能用point wolf正常运行一次 看看结果
好像还行，但是感觉也不是很明显的样子。。。。